### Load tensorflow

In [0]:
import tensorflow as tf
import numpy as numpy
import pandas as pandas
import os
tf.set_random_seed(1234)

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
pwd

u'/content'

In [0]:
data = pandas.read_csv('prices.csv')

### Check all columns in the dataset

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35532 entries, 0 to 35531
Data columns (total 7 columns):
date      35532 non-null object
symbol    35532 non-null object
open      35532 non-null float64
close     35532 non-null float64
low       35531 non-null float64
high      35531 non-null float64
volume    35531 non-null float64
dtypes: float64(5), object(2)
memory usage: 1.9+ MB


### Drop columns `date` and  `symbol`

In [0]:
data = data.drop('date', axis = 1)
data = data.drop('symbol', axis = 1)

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data_1000 = data.loc[0:999]

In [9]:
data_1000.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
Y = data_1000['close']
X = data_1000.drop('close', axis = 1)
trainX, testX,trainY,testY = train_test_split(X, Y, test_size=0.3, random_state=100)

In [12]:
trainY.shape

(700,)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [18]:
y_.shape

TensorShape([Dimension(None)])

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [20]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:trainX, 
                                                        y_:trainY})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 8324.612)
('Training loss at step: ', 10, ' is ', 6848.526)
('Training loss at step: ', 20, ' is ', 5766.571)
('Training loss at step: ', 30, ' is ', 3884.3477)
('Training loss at step: ', 40, ' is ', 3883.742)
('Training loss at step: ', 50, ' is ', 3883.6016)
('Training loss at step: ', 60, ' is ', 3883.5195)
('Training loss at step: ', 70, ' is ', 3883.4802)
('Training loss at step: ', 80, ' is ', 3883.4583)
('Training loss at step: ', 90, ' is ', 3883.4412)


In [0]:
_

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [22]:
sess.run(W)

array([[-2.224702 ],
       [-1.3972614],
       [ 5.8589606],
       [ 0.6460627]], dtype=float32)

In [23]:
sess.run(b)

array([24.586159], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [24]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op, loss],feed_dict={x:testX, 
                                                        y_:testY})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

('Testing loss at step: ', 0, ' is ', 2280.6519)
('Testing loss at step: ', 10, ' is ', 2280.4448)
('Testing loss at step: ', 20, ' is ', 2280.435)
('Testing loss at step: ', 30, ' is ', 2280.43)
('Testing loss at step: ', 40, ' is ', 2280.4248)
('Testing loss at step: ', 50, ' is ', 2280.42)
('Testing loss at step: ', 60, ' is ', 2280.417)
('Testing loss at step: ', 70, ' is ', 2280.4128)
('Testing loss at step: ', 80, ' is ', 2280.4104)
('Testing loss at step: ', 90, ' is ', 2280.4092)


In [25]:
test_loss

2280.4065

In [26]:
train_loss

3883.4304

In [27]:
train_loss - test_loss

1603.0239

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#set up learning rate 
sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer= sgd, loss='mse')

In [29]:
trainX.shape

(700, 4)

In [30]:
trainY.shape

(700,)

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Execute the model

In [32]:
model.fit(trainX,trainY,validation_data=(testX,testY), epochs=100)

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 0s 243us/step - loss: 3566.3877 - val_loss: 81056.6220
Epoch 2/100
700/700 [==============================] - 0s 35us/step - loss: 1267.2008 - val_loss: 560.8948
Epoch 3/100
700/700 [==============================] - 0s 32us/step - loss: 1756.9587 - val_loss: 1642.4248
Epoch 4/100
700/700 [==============================] - 0s 36us/step - loss: 1356.0361 - val_loss: 606.5466
Epoch 5/100
700/700 [==============================] - 0s 36us/step - loss: 1631.0839 - val_loss: 302.8553
Epoch 6/100
700/700 [==============================] - 0s 36us/step - loss: 2572.8125 - val_loss: 168.5830
Epoch 7/100
700/700 [==============================] - 0s 38us/step - loss: 1796.3339 - val_loss: 431.5862
Epoch 8/100
700/700 [==============================] - 0s 38us/step - loss: 1642.3133 - val_loss: 193.5594
Epoch 9/100
700/700 [==============================] - 0s 38us/step - loss: 1570.3302 - val_lo

In [33]:
model.get_weights()

[array([-0.9096455 , -0.01725314,  7.4280176 , -0.5275992 ], dtype=float32),
 array([ 0.565933  ,  0.00956313, -4.6292152 , -9.460036  ], dtype=float32),
 array([6.6545876e+01, 6.5857697e+01, 6.7222954e+01, 5.1630540e+06],
       dtype=float32),
 array([3.7718611e+03, 3.7082542e+03, 3.8093750e+03, 1.4416742e+14],
       dtype=float32),
 array([[-0.6970795 ],
        [-0.01350073],
        [ 5.6912413 ],
        [-7.822593  ]], dtype=float32),
 array([18.335089], dtype=float32)]

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Source = pandas.read_csv('Iris (3).csv')

In [35]:
Source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [0]:
Source = Source.drop('Id', axis=1)

### Splitting the data into feature set and target set

In [0]:
Y = Source['Species']
X = Source.drop('Species', axis = 1)

In [38]:
Y.head()


0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
Y_dummies = pandas.DataFrame(pandas.get_dummies(Y, sparse=True).values)

In [40]:
type(Y_dummies)

pandas.core.frame.DataFrame

In [53]:
Y_dummies.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [0]:
trainX, testX,trainY,testY = train_test_split(X, Y_dummies, test_size=0.3, random_state=100)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize the sequential model
model_iris = tf.keras.models.Sequential()

#Normalize the data
model_iris.add(tf.keras.layers.BatchNormalization())

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(10, activation='sigmoid', input_dim=4))

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Complile the model
model_iris.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Fitting the model and predicting 

In [46]:
model_iris.fit(trainX.values,trainY.values,validation_data=(testX,testY), epochs=1000)

Train on 105 samples, validate on 45 samples
Epoch 1/1000
105/105 [==============================] - 0s 2ms/step - loss: 1.1667 - acc: 0.3714 - val_loss: 1.2691 - val_acc: 0.2444
Epoch 2/1000
105/105 [==============================] - 0s 100us/step - loss: 1.1599 - acc: 0.3714 - val_loss: 1.2562 - val_acc: 0.2444
Epoch 3/1000
105/105 [==============================] - 0s 104us/step - loss: 1.1527 - acc: 0.3714 - val_loss: 1.2468 - val_acc: 0.2444
Epoch 4/1000
105/105 [==============================] - 0s 90us/step - loss: 1.1471 - acc: 0.3714 - val_loss: 1.2394 - val_acc: 0.2444
Epoch 5/1000
105/105 [==============================] - 0s 85us/step - loss: 1.1428 - acc: 0.3714 - val_loss: 1.2290 - val_acc: 0.2444
Epoch 6/1000
105/105 [==============================] - 0s 76us/step - loss: 1.1368 - acc: 0.3714 - val_loss: 1.2217 - val_acc: 0.2444
Epoch 7/1000
105/105 [==============================] - 0s 88us/step - loss: 1.1326 - acc: 0.3714 - val_loss: 1.2138 - val_acc: 0.2444
Epoch 8/1

In [47]:
model_iris.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch multiple                  16        
_________________________________________________________________
dense_1 (Dense)              multiple                  50        
_________________________________________________________________
dense_2 (Dense)              multiple                  88        
_________________________________________________________________
dense_3 (Dense)              multiple                  27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


In [48]:
model_iris.get_weights()

[array([1.3205292, 1.5055133, 1.7891656, 1.405719 ], dtype=float32),
 array([ 0.09343385,  0.13511747, -0.22417507, -0.2037996 ], dtype=float32),
 array([0., 0., 0., 0.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32),
 array([[ 0.12891398,  0.26681897, -0.36996704, -0.00922233, -0.23573215,
         -0.85345846, -0.26443598,  0.25622067, -0.6141776 , -0.39203316],
        [ 0.8248304 ,  0.24740723, -0.1047457 ,  0.583333  , -0.8266098 ,
          0.37202066, -0.6342222 ,  0.33109686, -0.61658734,  0.32854518],
        [-0.45940292, -0.40132332,  0.78181493,  0.5068294 ,  0.8438121 ,
         -0.50818235,  0.89026874, -0.6305408 ,  0.8479635 , -0.49814647],
        [-0.9185062 , -0.5464031 ,  0.35483956, -0.23513791,  0.04257697,
          0.5425221 ,  0.44580722, -0.185155  ,  0.7335181 , -0.3565643 ]],
       dtype=float32),
 array([ 0.05666274,  0.04909118, -0.05705774, -0.00688343, -0.05825923,
        -0.04472131, -0.05818095,  0.01473117, -0.09386516,  0.04119261],
     

### Report Accuracy of the predicted values

In [52]:
model_iris.evaluate(testX,testY)

45/45 [==============================] - 0s 80us/step


[0.4224045197168986, 1.0]